In [1]:
import pandas as pd   
import re
import numpy as np
import matplotlib.pyplot as plt
import os   
import scipy
from nltk.corpus import stopwords
from gensim.models import KeyedVectors

from nltk import pos_tag, word_tokenize
import nltk
nltk.download("punkt")
nltk.download('averaged_perceptron_tagger')
import torchtext.vocab as torch_vocab
import argparse
import torch


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Hezi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Hezi\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
model = torch_vocab.GloVe(name='twitter.27B',dim=100)
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [3]:
df = pd.read_csv("../../data/Paraphrase.csv")

In [4]:
df['glove_allwords'] = 0
df['glove_withoutstop'] = 0

In [5]:
df

,text_1,text_2,label,glove_allwords,glove_withoutstop
0,take measures in order to,take additional measures to,4.000,0,0
1,was seeking to,will strive to,3.000,0,0
2,canada aims to,canada works to,2.333,0,0
3,refugees and,refugee population,2.667,0,0
4,measures and,"instruments , and",3.000,0,0
...,...,...,...,...,...
993,unless specified otherwise,unless specifically stated otherwise,4.667,0,0
994,its efforts to combat,the area of combating,3.667,0,0
995,considered by the,give consideration to the,4.333,0,0
996,an enlarged security,an expanded security,4.000,0,0


In [18]:
def preprocess(raw_text,stopwords_remove=True):
    # keep only words
    letters_only_text = re.sub("[^a-zA-Z]", " ", str(raw_text))
    # convert to lower case and split 
    words = letters_only_text.lower().split()
    #print(words)
    # remove stopwords
    stopword_set = set(stopwords.words("english"))
    cleaned_words = list(set([w for w in words if w not in stopword_set]))
    return cleaned_words if stopwords_remove else words

def cosine_distance_wordembedding_method(s1, s2,stopwords_remove=True,remove_non_model=False):

    if remove_non_model:
        s1 = filter(lambda x: x in model.vocab, s1)
        s2 = filter(lambda x: x in model.vocab, s2)

    vector_1 = [model[word] for word in preprocess(s1,stopwords_remove)]
    vector_2 = [model[word] for word in preprocess(s2,stopwords_remove)]
    
    if len(vector_1)==0 or len(vector_2)==0:
        return -1
    
    #print(np.stack(vector_1))
    vector_1 = np.mean(np.stack(vector_1),axis=0)
    vector_2 = np.mean(np.stack(vector_2),axis=0)
    cosine = scipy.spatial.distance.cosine(vector_1, vector_2)
    return round((1-cosine)*100,2)


In [7]:
scores = []
for i in range(df.shape[0]):
    s1 = str(df['text_1'][i])
    s2 = str(df['text_2'][i])
    #print(s1,",",s2)
    dis = cosine_distance_wordembedding_method(s1,s2,stopwords_remove=False)
    df.loc[i,'glove_allwords']=dis

In [8]:
scores = []
for i in range(df.shape[0]):
    s1 = str(df['text_1'][i])
    s2 = str(df['text_2'][i])
    #print(s1,",",s2)
    dis = cosine_distance_wordembedding_method(s1,s2,stopwords_remove=True)
    df.loc[i,'glove_withoutstop']=dis

e:\WORK\ML\YDATA\NLP\assignment_research\nlprs\lib\site-packages\scipy\spatial\distance.py:720: RuntimeWarning: invalid value encountered in float_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [9]:
df.head(20)


,text_1,text_2,label,glove_allwords,glove_withoutstop
0,take measures in order to,take additional measures to,4.000,93.23,90.14
1,was seeking to,will strive to,3.000,86.77,41.71
2,canada aims to,canada works to,2.333,94.33,84.18
3,refugees and,refugee population,2.667,62.33,69.82
4,measures and,"instruments , and",3.000,82.54,33.48
5,provide both,", is provided",3.667,80.92,79.75
6,policy goals,political goals,4.333,93.09,93.09
7,of non-governmental organisations,for non-governmental organizations,4.000,93.95,97.76
8,of the service,of services are,3.333,93.22,85.51
9,resource availability .,of the resources available .,5.000,63.19,80.18


In [10]:

model = KeyedVectors.load_word2vec_format('e:\\WORK\\ML\\data\\embed\\fasttrack\\cc.en.300.vec')


In [14]:
df['ftext_allwords'] = 0
df['ftext_withoutstop'] = 0

In [20]:
scores = []
for i in range(df.shape[0]):
    s1 = str(df['text_1'][i])
    s2 = str(df['text_2'][i])
    #print(s1,",",s2)
    dis = cosine_distance_wordembedding_method(s1,s2,stopwords_remove=False,remove_non_model=True)
    df.loc[i,'ftext_allwords']=dis

In [21]:
scores = []
for i in range(df.shape[0]):
    s1 = str(df['text_1'][i])
    s2 = str(df['text_2'][i])
    #print(s1,",",s2)
    dis = cosine_distance_wordembedding_method(s1,s2,stopwords_remove=True,remove_non_model=True)
    df.loc[i,'ftext_withoutstop']=dis

In [23]:
df.head(20)

,text_1,text_2,label,glove_allwords,glove_withoutstop,ftext_allwords,ftext_withoutstop
0,take measures in order to,take additional measures to,4.000,93.23,90.14,84.92,89.26
1,was seeking to,will strive to,3.000,86.77,41.71,91.83,72.78
2,canada aims to,canada works to,2.333,94.33,84.18,91.88,77.32
3,refugees and,refugee population,2.667,62.33,69.82,94.71,89.21
4,measures and,"instruments , and",3.000,82.54,33.48,88.15,88.14
5,provide both,", is provided",3.667,80.92,79.75,89.44,73.50
6,policy goals,political goals,4.333,93.09,93.09,91.16,88.47
7,of non-governmental organisations,for non-governmental organizations,4.000,93.95,97.76,89.02,88.17
8,of the service,of services are,3.333,93.22,85.51,88.10,70.58
9,resource availability .,of the resources available .,5.000,63.19,80.18,91.14,84.40
